In [25]:
import pandas as pd

ratings1 = pd.read_csv('rating.csv')  # no necesitas sep='\t'
anime1 = pd.read_csv('anime.csv')

In [26]:
ratings = ratings1[['user_id', 'anime_id', 'rating']]
anime = anime1[['anime_id', 'name']]

### Observem el dataset

In [27]:
ratings.shape

(7813737, 3)

In [28]:
anime.shape

(12294, 2)

In [32]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [33]:
anime.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   anime_id  12294 non-null  int64 
 1   name      12294 non-null  object
dtypes: int64(1), object(1)
memory usage: 192.2+ KB


In [34]:
ratings['rating'].describe()

count    7.813737e+06
mean     6.144030e+00
std      3.727800e+00
min     -1.000000e+00
25%      6.000000e+00
50%      7.000000e+00
75%      9.000000e+00
max      1.000000e+01
Name: rating, dtype: float64

In [35]:
ratings['rating'].unique()

array([-1, 10,  8,  6,  9,  7,  3,  5,  4,  1,  2], dtype=int64)

In [37]:
ratings.isna().any()

user_id     False
anime_id    False
rating      False
dtype: bool

In [36]:
ratings['rating'].value_counts().sort_index()

rating
-1     1476496
 1       16649
 2       23150
 3       41453
 4      104291
 5      282806
 6      637775
 7     1375287
 8     1646019
 9     1254096
 10     955715
Name: count, dtype: int64

### Eliminem els -1

In [38]:
ratings = ratings[ratings['rating'] != -1]

In [40]:
ratings.shape

(6337241, 3)

### Duplicats

In [45]:
duplicates = ratings.duplicated(subset=['user_id', 'anime_id']).sum()
print(f"Número de duplicados: {duplicates}")

Número de duplicados: 7


In [46]:
duplicates2 = anime.duplicated(subset=['anime_id', 'name']).sum()
print(f"Número de duplicados: {duplicates2}")

Número de duplicados: 0


In [49]:
dup_mask = ratings.duplicated(subset=['user_id', 'anime_id'], keep=False)
ratings[dup_mask].sort_values(['user_id', 'anime_id']).head(20)

,user_id,anime_id,rating
4499243,42653,1575,8
4499286,42653,1575,6
4499244,42653,2001,8
4499288,42653,2001,10
4499255,42653,11757,8
4499307,42653,11757,5
4499258,42653,16498,8
4499316,42653,16498,8
4499261,42653,20507,7
4499320,42653,20507,9


In [51]:
dupes = ratings[dup_mask]
dupes.groupby(['user_id', 'anime_id'])['rating'].nunique().value_counts()

rating
2    6
1    1
Name: count, dtype: int64

In [19]:
ratings = pd.merge(anime, ratings)

In [20]:
ratings.head()

,anime_id,name,user_id,rating
0,32281,Kimi no Na wa.,99,5
1,32281,Kimi no Na wa.,152,10
2,32281,Kimi no Na wa.,244,10
3,32281,Kimi no Na wa.,271,10
4,32281,Kimi no Na wa.,278,-1
